This notebook extends the existing PyHolos $N_2O$ emissions calculator.

In [5]:
# Functions to compute N_20 - N_SNDirect 


import math

def compute_n2o_n_direct(N_SN_direct, N_CRN_direct, N_CRN_indirect, N_ON_direct):
    """
    Computes direct N₂O emissions from synthetic fertilizer application.
    """
    return N_SN_direct + N_CRN_direct + N_CRN_indirect + N_ON_direct


## Computing N_SN
def compute_N_fert_applied(product_N, product_P, product_K, content_product):
    """
    Computes N fertilizer applied when fertilizer information is available.
    """
    return (product_N + product_P + product_K) * (content_product / 100)

def compute_grainN_total(C_p, N_P):
    """
    Computes total nitrogen content in grains.
    """
    return (C_p / 0.45) * N_P

def compute_N_fert_applied_no_info(C_p, N_P, soil_test_N, fert_deposit, fert_efficiency, fixation=0.7, legume=0):
    """
    Computes N fertilizer applied when fertilizer information is NOT available.
    """
    if legume==0:  # Leguminous crop
        return (compute_grainN_total(C_p, N_P) * (1 - fixation) - soil_test_N - fert_deposit) / fert_efficiency
    else:  # Non-leguminous crop
        return (compute_grainN_total(C_p, N_P) - soil_test_N - fert_deposit) / fert_efficiency


def compute_N_SN(product_N, product_P, product_K, content_product, C_p, N_P, soil_test_N, fert_deposit, fert_efficiency, 
                 fert_info=1,fixation=0.7, legume=0):
    """
    Computes availability of N from synthetic fertilizer.
    """
    if fert_info==1:
        fert_applied = compute_N_fert_applied(product_N, product_P, product_K, content_product)
    else: # No fertilizer information
        fert_applied = compute_N_fert_applied_no_info(C_p, N_P, soil_test_N, fert_deposit, fert_efficiency, fixation, legume)

    return fert_applied + fert_deposit

############## COMPUTING EMISSION FACTOR
def compute_EF_CT(P_or_PE):
    """
    Computes EF_CT based on precipitation or evapotranspiration.
    """
    return math.exp(0.00558 * P_or_PE - 7.7)

def compute_EF_topo(P, PE, FR_Topo_i, irrigated=0):
    """
    Computes topographic emission factor based on precipitation and evapotranspiration.
    """
    if P > PE:
        return compute_EF_CT(P)
    elif P <= PE & irrigated==0: # If dry
        return (compute_EF_CT(PE) * FR_Topo_i) + (compute_EF_CT(P) * (1 - FR_Topo_i))
    else: # If irrigated
        return compute_EF_CT(PE)

def compute_EF_base(P, PE, FR_Topo_i, RF_TX_i, irrigated=0):
    """
    Computes base emission factor.
    """
    return compute_EF_topo(P, PE, FR_Topo_i, irrigated=0) * RF_TX_i / 0.645


def compute_EF(P, PE, FR_Topo_i, RF_TX_i, RF_NS_k, RF_Till_l, RF_CS_m, RF_AM, irrigated=0):
    """
    Computes emission factor. 
    """
    return compute_EF_base(P, PE, FR_Topo_i, RF_TX_i, irrigated=0,) * RF_NS_k * RF_Till_l * RF_CS_m * RF_AM


#### COMPUTING N20-NSN
def compute_N_SN_direct(product_N, product_P, product_K, content_product, C_p, N_P, soil_test_N, fert_deposit, 
                         RF_TX_i, RF_NS_k, RF_Till_l, RF_CS_m, RF_AM, P, PE, FR_Topo_i,
                         fert_efficiency=0.5, fixation=0.7, legume=0
                         , irrigated=0, fert_info=1):
    """
    Computes direct N₂O emissions from synthetic fertilizer application.
    """
    nsn = compute_N_SN(product_N, product_P, product_K, content_product, C_p, N_P, soil_test_N, fert_deposit, fert_efficiency, fert_info=1, fixation=0.7, legume=0)
    ef = compute_EF(P, PE, FR_Topo_i, RF_TX_i, RF_NS_k, RF_Till_l, RF_CS_m, RF_AM, irrigated=0)
    return nsn* ef 